## Imports

In [40]:
# Read stocks
import yfinance as yf

# For plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# To remove empty dates
import pandas as pd

# To calculate min max
import numpy as np

# To calculate TAs
import talib as ta

# For reading properties
from jproperties import Properties

## Load properties

In [41]:
configs = Properties()

with open('config/yf_ta_part1.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
START = configs.get('START').data
TEMPLATE  = configs.get('TEMPLATE').data
TICKERS = configs.get('TICKERS').data

In [42]:
ticker = yf.Ticker(TICKER)
df = ticker.history(start=START)[['Open', 'Close', 'Volume']]

## Utility methods

In [43]:
# Sets padding for figures
def set_padding(fig):
    fig.update_layout(margin=go.layout.Margin(
        r=10, #right margin
        b=10)) #bottom margin

# Adds the range selector to given figure
def add_range_selector(fig):
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=[
                    dict(count=1, label='1m', step='month', stepmode='backward'),
                    dict(count=6, label='6m', step='month', stepmode='backward'),
                    dict(count=1, label='YTD', step='year', stepmode='todate'),
                    dict(count=1, label='1y', step='year', stepmode='backward'),
                    dict(step='all')
                ]),
            type='date'),#end xaxis  definition
        xaxis2_type='date')

# Adds the volume chart to row 2, column 1
def add_volume_chart(fig):
    # Colours for the Bar chart
    colors = ['#9C1F0B' if row['Open'] - row['Close'] >= 0
          else '#2B8308' for index, row in df.iterrows()]

    # Adds the volume as a bar chart
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], showlegend=False, marker_color=colors), row=2, col=1)

## Remove Empty Dates
Reference: __[A Simple Guide to Plotly for Plotting Financial Chart](https://plainenglish.io/blog/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682)__

In [44]:
# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that are in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

## Plot Price and Volume charts

In [45]:
# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01, shared_xaxes=True)

# Plot the Price chart
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Price'), row=1, col=1)

# Add the volume chart
add_volume_chart(fig)

# Adds the range selector
add_range_selector(fig)

# Set the color from white to black on range selector buttons
fig.update_layout(xaxis=dict(rangeselector = dict(font = dict( color = 'black'))))

# Add labels to y axes
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)

# Sets customized padding
set_padding(fig)

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Set the template and the title
layout = go.Layout(template=TEMPLATE, title = TICKER + ' - Price and Volume', height=500, legend_title='Legend')
fig.update_layout(layout)

## Plot Price, SMA and EMA charts

In [46]:
df['SMA'] = ta.SMA(df['Close'],timeperiod=5)
df['EMA'] = ta.EMA(df['Close'], timeperiod = 5)

# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01, shared_xaxes=True)

# Plot the Price, SMA and EMA chart
for col in ['Close', 'SMA', 'EMA']: 
    fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col), row=1, col=1)
    
# Change the Close to Price for the lengend label
fig.data[0].name = 'Price'

# Add the volume chart
add_volume_chart(fig)

# Adds the range selector
add_range_selector(fig)

# Set the color from white to black on range selector buttons
fig.update_layout(xaxis=dict(rangeselector = dict(font = dict( color = 'black'))))

# Add labels to y axes
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)

# Sets customized padding
set_padding(fig)

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Set the template and the title
layout = go.Layout(template=TEMPLATE, title = TICKER + ' - Price, SMA, EMA and Volume', height=500, legend_title='Legend')
fig.update_layout(layout)

## Plot Price, SMA-50 and SMA-200 charts

In [47]:
df['SMA-50'] = ta.SMA(df['Close'],timeperiod=50)
df['SMA-200'] = ta.EMA(df['Close'], timeperiod =200)

# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01, shared_xaxes=True)

# Plot the Price, SMA and EMA chart
for col in ['Close', 'SMA-50', 'SMA-200']: 
    fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col), row=1, col=1)
    
# Change the Close to Price for the lengend label
fig.data[0].name = 'Price'

# Add the volume chart
add_volume_chart(fig)

# Adds the range selector
add_range_selector(fig)

# Set the color from white to black on range selector buttons
fig.update_layout(xaxis=dict(rangeselector = dict(font = dict( color = 'black'))))

# Add labels to y axes
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)

# Sets customized padding
set_padding(fig)

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Set the template and the title
layout = go.Layout(template=TEMPLATE, title = TICKER + ' - Price, SMA-50, SMA-200 and Volume', height=500, legend_title='Legend')
fig.update_layout(layout)

## Download multiple tickers

In [48]:
# List of tickers
ticker_list = TICKERS.split()

df_multi = yf.download(TICKERS, start=START)[['Close']]
df_multi.head()

[*********************100%***********************]  3 of 3 completed


Close                       
                  AAPL       GOOG        MSFT
Date                                         
2021-01-04  129.410004  86.412003  217.690002
2021-01-05  131.009995  87.045998  217.899994
2021-01-06  126.599998  86.764503  212.250000
2021-01-07  130.919998  89.362503  218.289993
2021-01-08  132.050003  90.360497  219.619995

## Normalize prices

In [49]:
# Normalize approach (x - min) / (max - min)

# Empty DF to collect normalized data
norm_df = pd.DataFrame()

# Copy the date column
norm_df.index = df.index

for ticker in ticker_list:
    max = np.max(df_multi['Close', ticker])
    min = np.min(df_multi['Close', ticker])
    norm_df[ticker + 'CloseN'] = np.array([(x - min) / (max - min) for x in df_multi['Close', ticker]])
norm_df.head()

,MSFTCloseN,GOOGCloseN,AAPLCloseN
Date,,,
2021-01-04 00:00:00-05:00,0.040044,0.043470,0.168148
2021-01-05 00:00:00-05:00,0.041590,0.052902,0.188764
2021-01-06 00:00:00-05:00,0.000000,0.048714,0.131942
2021-01-07 00:00:00-05:00,0.044461,0.087364,0.187605
2021-01-08 00:00:00-05:00,0.054251,0.102211,0.202165


## Plot Performance

In [50]:
# List to hold column names
column_names = []

# New names for the legend or else it will take column names
new_names = {}

for ticker in ticker_list:
    column_name = ticker + 'CloseN'
    column_names.append(column_name)
    new_names[column_name] = ticker

# PLot performance
fig = px.line(data_frame = norm_df, x = norm_df.index, y = column_names)

# Sets customized padding
set_padding(fig)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'),
            dict(count=3, label='3m', step='month', stepmode='backward'),
            dict(count=6, label='6m', step='month', stepmode='backward'),
            dict(count=1, label='1y', step='year', stepmode='backward'),
            dict(step='all')
        ])
    )
)
# Make it pretty and set the color from white to black on range selector buttons
layout = go.Layout(template=TEMPLATE, title='Performance', height=500,
                   xaxis=dict(rangeselector = dict(font = dict( color = 'black'))), legend_title='Legend')

# Change the legend names
fig.for_each_trace(lambda t: t.update(name = new_names[t.name]))
fig.update_layout(layout)